# **Import Library**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# **Download Dataset**

In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2023-12-10 10:06:47--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.200.207, 209.85.234.207, 142.251.171.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.200.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   202MB/s    in 0.7s    

2023-12-10 10:06:48 (202 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

--2023-12-10 10:06:48--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.200.207, 209.85.234.207, 142.251.171.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.200.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘validation-horse-or-human.zip’

validation-horse-or 100%[===================>]  10.95M  --.-KB/s    in 0.1s    

2023-12-10 10:06:48 (89.1 MB/s) - ‘validation-horse-or-human.zip’ saved [11480187/11480187]



# **Extract Dataset**

In [ ]:
import zipfile
import os

train_zip_dir = "/content/horse-or-human.zip"
val_zip_dir = "/content/validation-horse-or-human.zip"

train_dir = "/content/train"
val_dir = "/content/valid"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
with zipfile.ZipFile(train_zip_dir, mode='r') as zip_ref:
  zip_ref.extractall(train_dir)

with zipfile.ZipFile(val_zip_dir, mode='r') as zip_ref:
  zip_ref.extractall(val_dir)

In [ ]:
print(f'total training horse images: {len(os.listdir("/content/train/horses"))}')
print(f'total training human images: {len(os.listdir("/content/train/humans"))}')
print(f'total validation horse images: {len(os.listdir("/content/valid/horses"))}')
print(f'total validation human images: {len(os.listdir("/content/valid/humans"))}')

total training horse images: 500
total training human images: 527
total validation horse images: 128
total validation human images: 128


# **Data Preprocessing**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Augmentasi data gambar (contoh kali ini hanya melakukan resize, mengubah nilai ke tensor, dan normalisasi nilai pixel)
normalize_transform = transforms.Compose([transforms.Resize((300, 300)),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                           ])

# Make a dataset dari data yang diunduh dan sudah di-unzip
train_dataset = ImageFolder(train_dir, transform=normalize_transform)
valid_dataset = ImageFolder(val_dir, transform=normalize_transform)

batch_size_train = 128
batch_size_valid = 16

# Membuat data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size_valid, shuffle=False)

# **Make a Model**

In [ ]:
import torch.nn.functional as F

class CustomCNN(nn.Module):
  def __init__(self):
    super(CustomCNN, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=2, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(16, 32, kernel_size=2, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32, 32, kernel_size=2, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(128, 256, kernel_size=2, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        )
    self.flatten = nn.Flatten()
    self.fc = nn.Sequential(
        nn.Linear(256 * 3 * 3, 512),
        nn.ReLU(),
        nn.Linear(512, 1),
        nn.Sigmoid()
        )

  def forward(self, x):
    x = self.features(x)
    x = self.flatten(x)
    x = self.fc(x)
    return x

model = CustomCNN().to(device)

# Setting optimizer and loss function
optimizer = optim.RMSprop(params=model.parameters(), lr=0.0001)
loss_func = nn.BCELoss()

# **Training Model**

In [ ]:
epochs = 20
train_losses = []
train_accuracies = []
valid_accuracies = []

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct_train = 0
  total_train = 0
  for images, labels in train_loader:
    images, labels = images.to(device), labels.float().to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_func(outputs, labels.float().view(-1, 1))
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    predictions = (outputs > 0.5).float()
    correct_train += (predictions == labels.view(-1, 1)).sum().item()
    total_train += labels.size(0)

  train_loss = running_loss / len(train_loader)
  train_accuracy = correct_train / total_train
  train_losses.append(train_loss)
  train_accuracies.append(train_accuracy)

  model.eval()
  with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for images, labels in valid_loader:
      images, labels = images.to(device), labels.float().to(device)
      outputs = model(images)
      predictions = (outputs > 0.5).float()
      total_correct += (predictions == labels.view(-1, 1)).sum().item()
      total_samples += labels.size(0)

    valid_accuracy = total_correct / total_samples
    valid_accuracies.append(valid_accuracy)

  print(f'Epoch {epoch + 1}/{epochs}, '
  f'Train Loss: {train_loss:.4f}, '
  f'Train Accuracy: {train_accuracy:.4f}, '
  f'Validation Accuracy: {valid_accuracy:.4f}')

Epoch 1/20, Train Loss: 0.7055, Train Accuracy: 0.5355, Validation Accuracy: 0.5000
Epoch 2/20, Train Loss: 0.5989, Train Accuracy: 0.7196, Validation Accuracy: 0.5078
Epoch 3/20, Train Loss: 0.4374, Train Accuracy: 0.8121, Validation Accuracy: 0.6562
Epoch 4/20, Train Loss: 0.3205, Train Accuracy: 0.8578, Validation Accuracy: 0.6328
Epoch 5/20, Train Loss: 0.2980, Train Accuracy: 0.8676, Validation Accuracy: 0.8320
Epoch 6/20, Train Loss: 0.2521, Train Accuracy: 0.9133, Validation Accuracy: 0.7734
Epoch 7/20, Train Loss: 0.1969, Train Accuracy: 0.9124, Validation Accuracy: 0.8242
Epoch 8/20, Train Loss: 0.1482, Train Accuracy: 0.9348, Validation Accuracy: 0.8086
Epoch 9/20, Train Loss: 0.1469, Train Accuracy: 0.9357, Validation Accuracy: 0.8477
Epoch 10/20, Train Loss: 0.1201, Train Accuracy: 0.9542, Validation Accuracy: 0.8477
Epoch 11/20, Train Loss: 0.1257, Train Accuracy: 0.9630, Validation Accuracy: 0.8086
Epoch 12/20, Train Loss: 0.1219, Train Accuracy: 0.9581, Validation Accura

# **Predictions**

In [ ]:
from PIL import Image
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    path = "/content/" + fn
    img = Image.open(path).convert("RGB")
    img = normalize_transform(img).unsqueeze(0).to(device)
    output = model(img)
    prediction = (output > 0.5).item()

    if prediction:
        print(fn + " is a human")
    else:
        print(fn + " is a horse")

Saving WhatsApp Image 2022-01-19 at 6.10.44 AM.jpeg to WhatsApp Image 2022-01-19 at 6.10.44 AM.jpeg
WhatsApp Image 2022-01-19 at 6.10.44 AM.jpeg is a horse
